In [1]:
import torch
import dqc
import dqc.xc
import dqc.utils

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                return self.a * rho ** (self.p - 1)

In [3]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [51]:
mol = dqc.Mol(moldesc="N -1 0 0; N 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-54.0932, dtype=torch.float64, grad_fn=<AddBackward0>)


/home/radium/miniconda3/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.508e-02, |f|=2.288e-01 at iter 32
  warnings.warn(ConvergenceWarning(msg))


In [52]:
dm = qc._dm.detach().clone() # density matrix
dm

tensor([[ 4.4449e-02,  2.4137e-14,  1.0511e-01, -5.0424e-02,  1.0289e-01,
          2.2958e-14, -5.8054e-02,  8.7262e-15, -1.4612e-14, -4.7588e-14,
          1.2283e-01,  8.2387e-02,  1.9097e-01, -6.7452e-14,  3.2144e-02,
          1.8733e-14,  1.1592e-15,  1.0015e-13],
        [ 2.4137e-14,  9.9451e-02,  1.0472e-13, -6.4384e-14,  5.0526e-14,
         -1.8880e-02, -6.3688e-15,  1.1943e-06,  2.0303e-04, -1.8050e-01,
          1.5046e-13,  6.6087e-14,  2.0816e-13,  6.9983e-02, -9.8086e-14,
         -4.7394e-03,  1.2609e-03, -3.8878e-01],
        [ 1.0511e-01,  1.0472e-13,  4.0383e-01,  3.7858e-02,  1.9606e-01,
          3.0889e-14, -8.9278e-02,  5.3771e-14, -3.2877e-14, -9.8062e-14,
          1.4279e-01,  5.8679e-01,  4.2135e-01, -6.3006e-14, -2.0842e-01,
          6.0567e-14,  4.0853e-14,  8.9760e-14],
        [-5.0424e-02, -6.4384e-14,  3.7858e-02,  4.6050e-01, -9.5657e-02,
         -1.5449e-14,  4.3600e-02,  4.6571e-14,  4.2604e-14,  5.5748e-14,
         -6.9951e-02,  5.8823e-01, -5.7

In [53]:
coeff = qc.get_system().get_hamiltonian().dm2ao_orb_params(dm, 0) # full C matrix
coeff

tensor([[-1.0950e-01,  8.7311e-04,  7.6791e-02, -1.0924e-03, -5.1973e-02,
         -2.7058e-02,  3.0051e-02, -3.1940e-01,  7.3303e-01,  5.7967e-01,
         -1.3413e-03,  3.8196e-02,  8.9549e-04,  2.9453e-03, -1.0542e-02,
          5.3222e-04,  4.5250e-03,  2.8505e-02],
        [ 1.7530e-02,  1.3151e-03, -2.2185e-03, -8.9175e-04,  3.0709e-02,
          8.7984e-02,  2.0181e-01,  7.0415e-01,  1.3882e-02,  4.0309e-01,
          2.5778e-01, -1.8906e-01, -2.2763e-01,  3.0971e-02,  8.9148e-02,
         -3.9965e-02, -4.2306e-02, -3.5422e-01],
        [-1.5374e-01,  1.3291e-01,  1.8973e-01, -2.5090e-02, -1.7868e-01,
         -2.6067e-01,  1.5530e-01,  9.3562e-02, -3.6369e-01,  3.9657e-01,
         -9.6910e-02, -1.2887e-01, -5.6767e-02, -7.7344e-02, -7.0811e-02,
         -1.4887e-01, -6.6996e-02,  6.5797e-01],
        [ 1.7531e-01, -1.1001e-01, -2.3571e-01,  8.6145e-03,  1.6242e-02,
         -3.3918e-01,  1.2834e-01, -1.9639e-01, -1.4144e-01,  1.4508e-01,
         -3.7856e-01, -2.2579e-01, -2.6

In [54]:
fock = qc._engine.dm2scp(dm).clone()

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\textbf{C}\textbf{F}$$
$\textbf{f}$ here is matrix: $f_{ab}=\delta_{ab}f_a$

In [58]:
occ = torch.diag(qc._engine.orb_weight.detach().clone())
print(occ)

nao = qc.get_system().get_hamiltonian().nao
norb = qc._engine.norb
resize_functor = torch.nn.ZeroPad2d((0, nao-norb, 0, nao-norb))

occ = resize_functor(occ)
occ

tensor([[2., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 0., 0., 2.]], dtype=torch.float64)


tensor([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [56]:
derivative = torch.matmul(torch.matmul(occ, coeff), fock)
derivative

tensor([[-3.0361e-01,  2.6525e+00,  1.9177e-01, -2.7402e-02, -8.0257e-02,
         -2.5968e-02,  8.1371e-02, -1.0112e+00,  2.3214e+00,  3.1828e+00,
          3.5832e-01,  7.7232e-03, -4.8810e-02,  9.2791e-01,  7.3425e-02,
          1.6643e-01,  2.5242e-02, -2.3666e+00],
        [ 3.0112e-01,  1.6312e+00,  3.0017e-01, -5.3102e-02, -1.2828e+00,
          4.2610e-01,  2.0654e+00,  2.2497e+00,  6.0568e-02,  3.7278e+00,
          1.1565e+00, -3.1922e-01, -3.7798e-01,  1.7805e-01, -1.1387e+00,
         -1.8992e-05, -1.3473e-01, -4.5145e-01],
        [-1.6576e-01,  2.5521e+00,  6.8524e-01, -2.3596e-02, -2.1786e-01,
         -1.0441e+00,  4.4210e-01,  2.9998e-01, -1.1453e+00,  5.6567e-02,
          1.2838e+00, -3.5366e-01, -2.4048e-01,  1.2970e+00,  7.0034e-01,
         -1.1867e-01, -1.7175e-02, -3.6573e+00],
        [ 7.4620e-01,  2.5327e-02, -4.7102e-01,  1.4794e-01,  1.4825e+00,
         -8.4738e-01, -6.9070e-01, -5.9473e-01, -5.5114e-01,  1.5931e+00,
          3.2917e-01, -2.1184e-01,  1.2